In [104]:
def SplitData(data,M,k,seed):
    test=[]
    train=[]
    random.seed(seed)
    for u,m in data:
        if random.randint(0,M)==k:
            test.append([u,m])
        else:
            train.append([u,m])
    return train,test

def userSimilarity(train):
    W=dict()
    for u in train.keys():
        W[u]=dict()
        for v in train.keys():
            if u==v:
                continue
            W[u][v]=len(set(train[u].keys())&set(train[v].keys()))/math.sqrt(len(train[u])*len(train[v])*1.0)        
    return W

def UserSimilarity(train):
    #build inverse table for item_users
    item_users = dict()
    for u,items in train.items():
        for i in items.keys():
            if i not in item_users:
                item_users[i] = set()
            item_users[i].add(u)
            
    #calculate co-rated items between users
    C = dict()
    N = dict()
    for i, users in item_users.items():
        for u in users:
            if u not in C:
                C[u]=dict()
            if u not in N:
                N[u]=0
            N[u] +=1
            for v in users:
                if u==v:
                    continue
                if v not in C[u]:
                    C[u][v]=0
                C[u][v]+=1
    #calculate finial similarity matrix W
    W = dict()
    for u,related_users in C.items():
        W[u]=dict()
        for v,cuv in related_users.items():
            W[u][v] = cuv/math.sqrt(N[u] * N[v])
    return W


def UserSimilarity2(train):
    #build inverse table for item_users
    item_users = dict()
    for u,items in train.items():
        for i in items.keys():
            if i not in item_users:
                item_users[i] = set()
            item_users[i].add(u)
            
    #calculate co-rated items between users
    C = dict()
    N = dict()
    for i, users in item_users.items():
        for u in users:
            if u not in C:
                C[u]=dict()
            if u not in N:
                N[u]=0
            N[u] +=1
            for v in users:
                if u==v:
                    continue
                if v not in C[u]:
                    C[u][v]=0
                C[u][v]+=1/math.log(1+len(users))
    #calculate finial similarity matrix W
    W = dict()
    for u,related_users in C.items():
        W[u]=dict()
        for v,cuv in related_users.items():
            W[u][v] = cuv/math.sqrt(N[u] * N[v])
    return W

def Recommend(user,train,W):
    rank = dict()
    interacted_items = train[user]
    for v,wuv in sorted(W[user].items(),key=operator.itemgetter(1),reverse=True)[0:K]:
        for i , rvi in train[v].items():
            if i in interacted_items:
                continue
            if i not in rank:
                rank[i] = 0
#filter items user interacted                 
            rank[i] += wuv*rvi
    return rank

    
def Recall(train,test,N):
    hit = 0
    all = 0 
    for user in train.keys():
        tu = test[user] if test.get(user) else {}
        rank = GetRecommendation(user,N)
        for item,pui in rank:
            if item in tu:
                hit +=1
        all +=len(tu)
    return hit/(all * 1.0)

def Precision (train,test,N):
    hit = 0
    all = 0 
    for user in train.keys():
        tu = test[user] if test.get(user) else {}
        rank = GetRecommendation(user,N)
        for item,pui in rank:
            if tu and item in tu:
                hit +=1
        all += N
    return hit*1.0/(all*1.0)
    
def Coverage(train,test,N):
    recommend_items =set()
    all_items = set()
    for user in train.keys():
        for item in train[user].keys():
            all_items.add(item)
        rank = GetRecommendation(user,N)
        for item,pui in rank:
            recommend_items.add(item)
    return len(recommend_items)/(len(all_items)*1.0)

def Popularity(train, test, N):
    item_popularity = dict()
    for user, items in train.items():
        for item in items:
            if item not in item_popularity:
                item_popularity[item] = 0
            item_popularity[item] +=1
            
    ret = 0
    n = 0
    for user,items in train.items():
        rank = GetRecommendation(user,N)
        for item,pui in rank:
            ret += math.log(1+item_popularity[item])
            n+=1
    ret /= n*1.0
    return ret

def loadData(inputPath):
    f = open(inputPath)
    data=[]
    line=f.readline()
    while (line):
        data.append(line.split('\t')[:2])
        line = f.readline()
    return data

def toDict(data):
    d=dict()
    for u,i in data:
        if u not in d:
            d[u]=dict()
        if i not in d[u]:
            d[u][i]=0
        d[u][i]+=1
    return d

def GetRecommendation(user,N):
    rank = Recommend(user,train,W)
    topN = sorted(rank.items(),key=operator.itemgetter(1),reverse=True)[0:N]
    return topN

import random,math,operator
# data = {'A':{'a':1,'b':1,'d':1},'B':{'a':1,'c':1},'C':{'b':1,'e':1},'D':{'c':1,'d':1,'e':1}}

# print 'A:',Recommend('A',data,W)
inputPath = "/home/yanbin/data/ml-100k/u.data"
data = loadData(inputPath)
train,test = SplitData(data,5,0,40)

train = toDict(train)
test = toDict(test)
W = UserSimilarity(train)
GetRecommendation('2',10)
# Recommend('1',data,W)

# Popularity(data,data,N)
Ks=[5,10,20,40,80,160]
N=10
for k in Ks:
    K=k
    recall = Recall(train,test,N)
    precision = Precision(train,test,N)
    coverage = Coverage(train,test,N)
    popularity = Popularity(train,test,N)
    print ("k:%s,准确率：%2.2f%%，召回率:%2.2f%%，覆盖率:%2.2f%%，流行度:%2.2f%%" 
        %(K,precision*100,recall*100,coverage*100,popularity*100))

k:5,准确率：22.22%，召回率:12.52%，覆盖率:40.19%，流行度:517.16%
k:10,准确率：25.54%，召回率:14.40%，覆盖率:30.30%，流行度:531.51%
k:20,准确率：28.13%，召回率:15.86%，覆盖率:24.20%，流行度:542.29%
k:40,准确率：29.09%，召回率:16.40%，覆盖率:18.17%，流行度:550.37%
k:80,准确率：29.07%，召回率:16.39%，覆盖率:13.22%，流行度:556.56%
k:160,准确率：27.94%，召回率:15.75%，覆盖率:10.38%，流行度:561.75%


In [101]:
W = UserSimilarity2(train)
GetRecommendation('2',10)
# Recommend('1',data,W)

# Popularity(data,data,N)
Ks=[5,10,20,40,80,160]
N=10
for k in Ks:
    K=k
    recall = Recall(train,test,N)
    precision = Precision(train,test,N)
    coverage = Coverage(train,test,N)
    popularity = Popularity(train,test,N)
    print ("k:%s,准确率：%2.2f%%，召回率:%2.2f%%，覆盖率:%2.2f%%，流行度:%2.2f%%" 
        %(K,precision*100,recall*100,coverage*100,popularity*100))

1289 9430
k:5,准确率：13.67%，召回率:14.26%，覆盖率:42.54%，流行度:518.91%
1593 9430
k:10,准确率：16.89%，召回率:17.62%，覆盖率:32.65%，流行度:535.19%
1736 9430
k:20,准确率：18.41%，召回率:19.20%，覆盖率:26.12%，流行度:546.35%
1788 9430
k:40,准确率：18.96%，召回率:19.78%，覆盖率:20.01%，流行度:554.69%
1761 9430
k:80,准确率：18.67%，召回率:19.48%，覆盖率:15.52%，流行度:561.41%
1667 9430
k:160,准确率：17.68%，召回率:18.44%，覆盖率:11.98%，流行度:566.97%


In [103]:
def topN(train,N):
    items_times=dict()
    for user,items in train.items():
        for item,times in items.items():
            if item not in items_times:
                items_times[item]=0
            items_times[item]+=1
    return sorted(items_times.items(),key=operator.itemgetter(1),reverse=True)[0:N]
def GetRecommendation(user,N):
    return topN(train,N)
Ks=[5]
N=10
topNs=topN(train,N)
for k in Ks:
    K=k
    recall = Recall(train,test,N)
    precision = Precision(train,test,N)
    coverage = Coverage(train,test,N)
    popularity = Popularity(train,test,N)
    print ("k:%s,准确率：%2.2f%%，召回率:%2.2f%%，覆盖率:%2.2f%%，流行度:%2.2f%%" 
        %(K,precision*100,recall*100,coverage*100,popularity*100))

def Random(train,N):
    result=[]
    for i in range(N):
        rd=random.randint(0,len(train)-1)
        result.append([train[rd][-1],0])
    return result

def GetRecommendation(user,N):
    return Random(train2,N)
data = loadData(inputPath)
train2,test = SplitData(data,10,0,40)
train = toDict(train2)
test = toDict(test)
Ks=[5,10,20,40,80,160]
N=10
for k in Ks:
    K=k
    recall = Recall(train,test,N)
    precision = Precision(train,test,N)
    coverage = Coverage(train,test,N)
    popularity = Popularity(train,test,N)
    print ("k:%s,准确率：%2.2f%%，召回率:%2.2f%%，覆盖率:%2.2f%%，流行度:%2.2f%%" 
        %(K,precision*100,recall*100,coverage*100,popularity*100))


451 9430
k:5,准确率：4.78%，召回率:4.99%，覆盖率:0.60%，流行度:608.60%
144 9430
k:5,准确率：1.53%，召回率:1.92%，覆盖率:73.64%，流行度:471.00%
144 9430
k:10,准确率：1.53%，召回率:1.78%，覆盖率:73.22%，流行度:469.87%
150 9430
k:20,准确率：1.59%，召回率:1.67%，覆盖率:72.56%，流行度:469.83%
150 9430
k:40,准确率：1.59%，召回率:1.66%，覆盖率:71.96%，流行度:471.80%
135 9430
k:80,准确率：1.43%，召回率:1.39%，覆盖率:72.74%，流行度:470.88%
146 9430
k:160,准确率：1.55%，召回率:1.75%，覆盖率:72.98%，流行度:469.58%
